In [1]:
import pandas as pd
import numpy as np
import pyspark
from pydataset import data
from pyspark.sql.functions import sum, mean, concat, lit, regexp_extract, regexp_replace, when
from vega_datasets import data
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import *

np.random.seed(123)

In [2]:
# start spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

## 1.Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
pandas_dataframe = pd.DataFrame(
    {
        "language": np.random.choice(list(['Python', 'SQL','React','Java']), 20)
    }
)

In [4]:
# create spark df and show first 5 records
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+--------+
|language|
+--------+
|   React|
|     SQL|
|   React|
|   React|
|  Python|
+--------+
only showing top 5 rows



In [5]:
# view schema of df
df.printSchema()

root
 |-- language: string (nullable = true)



In [6]:
# get shape of df
print((df.count(), len(df.columns)))

(20, 1)


## 2. Load the mpg dataset as a spark dataframe.

In [6]:
from pydataset import data


In [7]:
mpg = spark.createDataFrame(data('mpg'))

In [8]:
mpg.head()

Row(manufacturer='audi', model='a4', displ=1.8, year=1999, cyl=4, trans='auto(l5)', drv='f', cty=18, hwy=29, fl='p', class='compact')

### a. Create 1 column of output that contains a message like the one below for each vehicle:

The 1999 audi a4 has a 4 cylinder engine.


In [9]:
mpg.select(
    concat(
        lit("The "),
        col("year"),
        lit(" "),
        col("manufacturer"),
        lit(" "),
        col("model"),
        lit(" has a "),
        col("cyl"),
        lit(" cylinder engine."),
    ).alias("vehicle_cylinder_desc")
).show(truncate=False)

+--------------------------------------------------------------+
|vehicle_cylinder_desc                                         |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

### b. Transform the trans column so that it only contains either manual or auto.

In [10]:
mpg = mpg.select("*", mpg.trans, regexp_extract("trans", r"(.\w+)", 1).alias('transformed_trans'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|     trans|transformed_trans|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|  auto(l5)|             auto|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|manual(m5)|           manual|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|manual(m6)|           manual|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|  auto(av)|             auto|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|  auto(l5)|             auto|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
only showing top 5 rows



## 3. Load the tips dataset as a spark dataframe.

In [11]:
tips = spark.createDataFrame(data('tips'))

### a. What percentage of observations are smokers?

In [ ]:
tips.show(3)

In [ ]:
tips.createOrReplaceTempView("tips")

In [ ]:
spark.sql(
    """
SELECT ((SELECT COUNT(smoker)
FROM tips
WHERE smoker = 'Yes') / 
(SELECT COUNT(smoker)
FROM tips)*100) as pct_smokers
FROM tips
"""
).show(1)

### b. Create a column that contains the tip percentage

In [ ]:
col = tips.tip / tips.total_bill
df_with_tip_pct = tips.select('*', col.alias('tip_pct'))

In [ ]:
df_with_tip_pct.show(3)

### c. Calculate the average tip percentage for each combination of sex and smoker.


In [ ]:
df_with_tip_pct.groupby("sex").pivot('smoker').mean('tip_pct').show()

## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.



In [9]:
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



### a. Convert the temperatures to fahrenheit.

In [10]:
weather = weather.withColumn('temp_max_F', ((weather.temp_max.cast('double') * 9 / 5) + 32)).drop('temp_max')
weather = weather.withColumn('temp_min_F', ((weather.temp_min.cast('double') * 9 / 5) + 32)).drop('temp_min')
weather.show(5)

+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_F|temp_min_F|
+----------+-------------+----+-------+----------+----------+
|2012-01-01|          0.0| 4.7|drizzle|     55.04|      41.0|
|2012-01-02|         10.9| 4.5|   rain|     51.08|     37.04|
|2012-01-03|          0.8| 2.3|   rain|     53.06|     44.96|
|2012-01-04|         20.3| 4.7|   rain|     53.96|     42.08|
|2012-01-05|          1.3| 6.1|   rain|     48.02|     37.04|
+----------+-------------+----+-------+----------+----------+
only showing top 5 rows



### b. Which month has the most rain, on average?

In [20]:
(weather.withColumn('month', month('date'))
    .groupBy('month')
    .agg(mean('precipitation').alias('avg_rain'))
    .sort(desc('avg_rain'))
    .show(1))

+-----+-----------------+
|month|         avg_rain|
+-----+-----------------+
|   11|5.354166666666667|
+-----+-----------------+
only showing top 1 row



### c. Which year was the windiest?

In [21]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_winds"))
    .sort(col("total_winds").desc())
    .head(5)
)

[Row(year=2012, total_winds=1244.7),
 Row(year=2014, total_winds=1236.5000000000007),
 Row(year=2015, total_winds=1153.3000000000002),
 Row(year=2013, total_winds=1100.8000000000006)]

### d. What is the most frequent type of weather in January?

In [22]:
(
    weather.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



### e. What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [24]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max_F").alias("average_high_temp"),
        avg("temp_min_F").alias("average_low_temp"),
    )
    .show()
)


+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



### f. What percentage of days were rainy in q3 of 2015?

In [25]:
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



### g. For each year, find what percentage of days it rained (had non-zero precipitation).

In [26]:
(
    weather.withColumn("year", year("date"))
    .select(when(col("precipitation") > 0, 1).otherwise(0).alias("rain"), "year")
    .groupby("year")
    .agg(mean("rain"))
    .show()
)

+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2015|0.39452054794520547|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2012|0.48360655737704916|
+----+-------------------+

